### Training the model

In [1]:
import IPython
from classes.vqa_model_trainer import VqaModelTrainer
from common.model_utils import get_trainable_params_distribution
from common.functions import get_highlighted_function_code

Using TensorFlow backend.


In [2]:
import logging
from vqa_logger import init_log
init_log()
logger = logging.getLogger(__name__)

In [3]:
model_location = 'C:\\Users\\Public\\Documents\\Data\\2018\\vqa_models\\20190125_1052_13\\vqa_model_.h5'

### Loading the model to train:

In [4]:
epochs = 1
batch_size = 75
mt = VqaModelTrainer(model_location, use_augmentation=True,batch_size=batch_size)

[2019-01-25 15:14:48][DEBUG] Loading Model: 0:00:03.102193
[2019-01-25 15:14:48][INFO] Using specs from: "C:\Users\Public\Documents\Data\2018\vqa_models\20190125_1052_13\vqa_specs.pkl"
[2019-01-25 15:14:48][DEBUG] Loading Meta: 0:00:00.125360
[2019-01-25 15:14:48][DEBUG] Loading the data from C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.h5
[2019-01-25 15:14:55][DEBUG] Loading Data: 0:00:06.355840


#### Lets take a look at the parameters:

In [5]:
get_trainable_params_distribution(mt.model)
# mt.model.summary()

Got a total of 362,180 trainable parameters


,index,layer,trainable_params,pretty_value
0,4,model_output_tanh_dense/kernel:0,303360,"303,360"
1,5,dense_1/kernel:0,36864,"36,864"
2,10,embbeding_LSTM/recurrent_kernel:0,16384,"16,384"
3,3,model_output_tanh_dense/bias:0,4740,"4,740"


#### And a look at data:

In [6]:
mt.df_meta_answers
mt.df_meta_words
mt.df_meta_imaging_devices
mt.df_meta_answers.tail(2)

,answer
4738,type ii optic nerve
4739,ct scan bra image


In [7]:
logger.debug(f"train Shape: {mt.data_train.shape}")
logger.debug(f"validation Shape: {mt.data_val.shape}")
mt.data_train.head(0)

[2019-01-25 15:14:55][DEBUG] train Shape: (5413, 19)
[2019-01-25 15:14:55][DEBUG] validation Shape: (500, 19)


,level_0,index,image_name,question,answer,group,path,original_question,original_answer,tumor,hematoma,brain,abdomen,neck,liver,imaging_device,answer_embedding,question_embedding,is_imaging_device_question


### Overview of preperations for training:

##### The functions for getting the features & labels:

In [8]:
from common.functions import get_features, concat_row, sentences_to_hot_vector, hot_vector_to_words
code_get_labels = get_highlighted_function_code(mt.get_labels, remove_comments=True)


code_get_features = get_highlighted_function_code(get_features, remove_comments=True)
code_concat = get_highlighted_function_code(concat_row, remove_comments=True)
code_hot_vector = get_highlighted_function_code(sentences_to_hot_vector, remove_comments=True)


print('Getting the label using a hot vector\n')
IPython.display.display(code_get_labels)
print('\n\nThe underlying method:\n')
IPython.display.display(code_hot_vector)


print('\n\nGetting the features using question embeding concatenation:\n')
IPython.display.display(code_get_features)
IPython.display.display(code_concat)




Getting the label using a hot vector





The underlying method:





Getting the features using question embeding concatenation:



##### Example of hot vector of answer (AKA answer...)

In [9]:
df = mt.data_train

class_df = mt.class_df
class_count = len(class_df)
# class_df

classes_indices_df = [class_df.loc[class_df.isin(ans.lower().split())] for ans in  df.answer]
classes_indices = [list(d.index) for d in classes_indices_df]

idx_sample = 9
print(df.answer[idx_sample])
classes_indices[idx_sample]

mri


[2062]

##### Will transform the sentences into vector and back using the following:

In [10]:
code = get_highlighted_function_code(hot_vector_to_words,remove_comments=False)
IPython.display.display(code)  

##### Check it looks sane by inversing the binarizing:

In [11]:
# words = mt.df_meta_words.word

arr_one_hot_vector = mt.get_labels(mt.data_train)
categorial_labels = arr_one_hot_vector

idx = 0
answer =  mt.data_train.answer.loc[idx]
print(f'The sentence:\n{answer}')

one_hot_vector = arr_one_hot_vector[idx]
label_words = hot_vector_to_words(one_hot_vector, mt.class_df)
print('\n\nThe reversed answer from hot vector:')
label_words

[2019-01-25 15:14:57][DEBUG] Using single label


The sentence:
tumor at tail pancreas


The reversed answer from hot vector:


266    tumor at tail pancreas
Name: answer, dtype: object

In [31]:
history = mt.train()

[2019-01-25 15:21:25][DEBUG] Getting train features: 0:00:02.880118
[2019-01-25 15:21:25][DEBUG] Using single label
[2019-01-25 15:21:25][DEBUG] Getting validation labels: 0:00:00.027382


Len: 75


[2019-01-25 15:21:35][DEBUG] Getting train features: 0:00:00.362348
[2019-01-25 15:21:35][DEBUG] Using multi label
[2019-01-25 15:21:35][DEBUG] Getting train labels: 0:00:00.038263
[2019-01-25 15:21:35][DEBUG] Expected shape: [(None, 12288, 1), (None, None, None, 3)]
[2019-01-25 15:21:35][DEBUG] ---------------------------------------------------------------------------
[2019-01-25 15:21:35][DEBUG] Actual training shape:((75, 12288, 1), (75, 224, 224, 3))
[2019-01-25 15:21:35][DEBUG] Train Labels shape:(75, 3389)
[2019-01-25 15:21:35][DEBUG] ---------------------------------------------------------------------------
[2019-01-25 15:21:35][DEBUG] Actual Validation shape:((500, 12288, 1), (500, 224, 224, 3))
[2019-01-25 15:21:35][DEBUG] Validation Labels shape:(500, 4740)


Epoch 1/1


[2019-01-25 15:22:14][DEBUG] Training Model: 0:00:48.762536
[2019-01-25 15:22:14][ERROR] Got an error training model: Error when checking target: expected model_output_tanh_dense to have shape (4740,) but got array with shape (3389,)


ValueError: Error when checking target: expected model_output_tanh_dense to have shape (4740,) but got array with shape (3389,)

### Save trained model:

In [ ]:
with VerboseTimer("Saving trained Model"):
    model_fn, summary_fn, fn_image, fn_history = mt.save(mt.model, history)


In [ ]:
print (model_fn)